# Adding Cloud-Optimized GeoTIFFs to the MAAP Dashboard

The following notebook steps through how to add a dataset to the MAAP Dashboard.

The MAAP dashboard has 3 environments:

* dit: biomass.dit.maap-project.org
* staging: biomass.staging.maap-project.org
* production: biomass.maap-project.org

These instructions will guide you towards adding your dataset to `bimoass.dit.maap-project.org`. The MAAP Dashboard team will "promote" changes to staging and production periodically (release schedule forthcoming).

Pre-requisites:

* jupyter notebook
* rasterio, rio-cogeo, supermercado

In [ ]:
%%capture
!pip install rasterio rio-cogeo supermercado

In [ ]:
# import the MAAP package
from maap.maap import MAAP
import json
import os
import matplotlib

# create MAAP class
maap = MAAP(maap_host='api.ops.maap-project.org')

## Check your file is a valid COG

In [ ]:
my_tif = '5_biomass_cog.masked.tif'
project_dir = "/projects"

In [ ]:
%%bash -s "$project_dir" "$my_tif"
rio cogeo validate $1/$2

## Upload the file

In [ ]:
## Helper functions
def generate_s3_upload_path(uuid, file):
    filename = os.path.basename(file)
    bucket = maap.config['aws']['user_upload_bucket']
    directory = maap.config['aws']['user_upload_directory']
    location = f"s3://{bucket}/{directory}/{uuid}/{filename}"
    return location

def upload_and_return_location(files):
    response = maap.uploadFiles(files)
    uuid = response\
      .replace("Upload file subdirectory: ", "")\
      .replace(" (keep a record of this if you want to share these files with other users)", "")
    return [generate_s3_upload_path(uuid, file) for file in files]

location = upload_and_return_location([f"{project_dir}/{my_tif}"])[0]
s3_path = location.replace('s3://', '')
location

# Identify the right rescale and color parameters

In [ ]:
%%bash -s "$s3_path"
gdalinfo /vsis3/$1 -stats

## Get valid x, y parameters for a given zoom

When generating tiles, the titiler API requires a valid x, y set for the given zoom.

In [ ]:
%%bash -s "$location"
rio bounds $1 | supermercado burn 1 # this last value is the "zoom"

# Test the link with the titiler

In [ ]:
band_min=0
band_max=140
zxy = '1/0/1'
titiler_url = f"https://titiler.maap-project.org"
rescale = f"{band_min},{band_max}"
band_index = 1
test_img_url = f"{titiler_url}/cog/tiles/{zxy}.png?url={location}&rescale={rescale}&bidx={band_index}"
# Jupyter auto adds &amp; to links so copy / paste everything after "x" into a browser
print(f"x{test_img_url}")

# Define a color map

By default, the image will be displayed in greyscale if no `colormap_name` parameter is passed to the titiler API. Guidance below is provided to help determine what a valid colormap_name might be and how to create a legend for the dashboard.

## Dashboard ColorRamps & Legends

When using the dashboard, there 2 components for implementing a color scheme for your map. There is the map render and there is the legend.

> Titiler used for Cloud Optimized Geotiff (COG) rendering accepts any color scheme from the python matplotlib library, and custom color formulas.

* [Rio Tiler Colors](https://cogeotiff.github.io/rio-tiler/colormap/)
* [Matplotlib Colors](https://matplotlib.org/stable/tutorials/colors/colormaps.html) 

Available `colormap_name` values for titiler: `above, accent, accent_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, blues, blues_r, bone, bone_r, brbg, brbg_r, brg, brg_r, bugn, bugn_r, bupu, bupu_r, bwr, bwr_r, cfastie, cividis, cividis_r, cmrmap, cmrmap_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, dark2, dark2_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnbu, gnbu_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, greens, greens_r, greys, greys_r, hot, hot_r, hsv, hsv_r, inferno, inferno_r, jet, jet_r, magma, magma_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, oranges, oranges_r, orrd, orrd_r, paired, paired_r, pastel1, pastel1_r, pastel2, pastel2_r, pink, pink_r, piyg, piyg_r, plasma, plasma_r, prgn, prgn_r, prism, prism_r, pubu, pubu_r, pubugn, pubugn_r, puor, puor_r, purd, purd_r, purples, purples_r, rainbow, rainbow_r, rdbu, rdbu_r, rdgy, rdgy_r, rdpu, rdpu_r, rdylbu, rdylbu_r, rdylgn, rdylgn_r, reds, reds_r, rplumbo, schwarzwald, seismic, seismic_r, set1, set1_r, set2, set2_r, set3, set3_r, spectral, spectral_r, spring, spring_r, summer, summer_r, tab10, tab10_r, tab20, tab20_r, tab20b, tab20b_r, tab20c, tab20c_r, terrain, terrain_r, twilight, twilight_r, twilight_shifted, twilight_shifted_r, viridis, viridis_r, winter, winter_r, wistia, wistia_r, ylgn, ylgn_r, ylgnbu, ylgnbu_r, ylorbr, ylorbr_r, ylorrd, ylorrd_r`


## Example 1: Class based known colors

In this example, the raster represents classes of forest with 11 possible values. There are specific colors selected to correspond to each class. We combine the list of colors and the list of classes and format them for the legend parameter the dashboard needs.

https://github.com/MAAP-Project/dashboard-datasets-maap/blob/main/datasets/taiga-forest-classification.json

In [ ]:
colors = [
    '#5255A3','#1796A3','#FDBF6F','#FF7F00', '#FFFFBF','#D9EF8B','#91CF60','#1A9850', '#C4C4C4','#FF0000', '#0000FF'
]
labels = [
    'Sparse & Uniform',
    'Sparse & Diffuse-gradual',
    'Sparse & Diffuse-rapid',
    'Sparse & Abrupt ',
    'Open & Uniform ',
    'Open & Diffuse-gradual',
    'Open & Diffuse-rapid',
    'Open & Abrupt',
    'Intermediate & Closed',
    'Non-forest edge (dry)',
    'Non-forest edge (wet)'
]

legend = [dict(color=colors[i],label=labels[i]) for i in range(0, len(colors))]
print(json.dumps(legend, indent=2))

# Copy and Paste the output below to your dashboard config.

## Example 2: Discrete ColorRamp

In this example, the range of values is known, but the color scale has many non-sequential colors. Starting with the premade color list, we create a continuous color ramp that uses the known colors as stops points. Arbitrarly 12 breaks looked decent in the dashboard legend so we split it into 12 discrete colors. Then combine the list of values and colors into the correct json syntax.

https://github.com/MAAP-Project/dashboard-datasets-maap/blob/main/datasets/ATL08.json

In [ ]:
forest_ht = matplotlib.colors.LinearSegmentedColormap.from_list('forest_ht', ['#636363','#FC8D59','#FEE08B','#FFFFBF','#D9EF8B','#91CF60','#1A9850','#005A32'], 12)
cols = [matplotlib.colors.to_hex(forest_ht(i)) for i in range(forest_ht.N)]

cats = range(0,25, (25//len(cols)))
legend = [[cats[i],cols[i]] for i in range(0, len(cols))]
text = (json.dumps(legend, separators=(',', ': ') ))

print(text.replace('],[','],\n['))
 
# Copy and Paste the output below to your dashboard config.

## Example 3: Continuous ColorRamp

In this example, we are using a built in ColorRamp from matplotlib. So we just need to extract enough colors to fill the legend adequately, and convert the colors to hex codes.

https://github.com/MAAP-Project/dashboard-datasets-maap/blob/main/datasets/topo.json

In [ ]:
cmap_name = 'viridis'
cmap = matplotlib.cm.get_cmap(cmap_name, 12)
cols = [matplotlib.colors.to_hex(cmap(i)) for i in range(cmap.N)]
print(cols)

# Copy and Paste the output below to your dashboard config.

# Create and submit your dashboard dataset json

In [ ]:
# This example is for a continuous color ramps
dataset_id = "paraguay-estimated-biomass"
dataset_name = "Estimated Biomass in Paraguay"
dataset_type = "raster"
legend_type = "gradient-adjustable"
info = "Estimated biomass within 6km grids."
stops = cols
tiles_link = f"{{titiler_server_url}}/cog/tiles/{{z}}/{{x}}/{{y}}.png?url={location}&colormap_name={cmap_name}&rescale={band_min},{band_max}&bidx={band_index}"

In [ ]:
dataset_dict = {
    "id": dataset_id,
    "name": dataset_name,
    "type": dataset_type,
    "swatch": {
      "color": "#6976d7",
      "name": "Moody Blue"
    },
    "source": {
        "type": dataset_type,
        "tiles": [ tiles_link ]
    },
    "legend": {
      "type": legend_type,
       "min": band_min,
       "max": band_max,
      "stops": stops
    },
    "info": info
}
print(json.dumps(dataset_dict, indent=4))

## Create a PR to the datasets repo

```bash
git clone git@github.com:MAAP-Project/biomass-dashboard-datasets.git
cd biomass-dashboard-datasets
git checkout -b ab/add-dataset
# select and copy json above
echo <copied_json> >> datasets/paraguay-estimated-biomass.json
```

### Add to the datasets list in config.yml

In `config.yml`:

```yaml
DATASETS:
- paraguay-estimated-biomass.json
```

### Add to the product or country pilot

In `country_pilots/paraguay/country_pilot.json`:
```json
{
    "id": "paraguay",
    "label": "Paraguay",
    //...
    "datasets": [
        {
            "id": "paraguay-forest-mask"
        },
        {
            "id": "paraguay-tree-cover"
        },
        {
            "id": "paraguay-estimated-biomass"
        }
    ]
}
```

## Add content to the summary

There should be a `summary.html` file corresponding to the product or country pilot you are working on, for example: `country_pilots/paraguay/summary.html`. Add or modify content in that file as appropriate.

## Submit a PR

Once you have added the dataset json file and summary content, submit a PR to https://github.com/MAAP-Project/biomass-dashboard-datasets. A member of the data team will review the PR and when it is merged your content will appear in biomass.dit.maap-project.org.

# Optional: Create a mosaic

Many datasets are comprised of many tiles distributed spatially over the globe. In order to visualize them all together, we can use [mosaicJSON](https://github.com/developmentseed/mosaicjson-spec) to create a mosaic for the dynamic tiler API. The dynamic tiler API knows how to read this mosaicJSON and select which tiles to render based on the current zoom, x and y coordinates across spatially distinct COGs.

In [ ]:
import glob
import os
import urllib

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [ ]:
%%capture
!pip install cogeo-mosaic

In [ ]:
# Local Path to your COGs
output = "/projects/xXx/XXX/xxx/"

# Search for files to include, use recursive if nested folders (common in DPS output)
files = glob.glob(os.path.join(output, "*.tif"), recursive=False)
files = [os.path.basename(f) for f in files]
files

In [ ]:
# Skipping this step since we don't want to upload these large files more than once!
# maap.uploadFiles(files)

In [ ]:
uuid = ''
tiles = [generate_s3_upload_path(uuid, file) for file in files]
print(tiles)

In [ ]:
mosaicdata = MosaicJSON.from_urls(tiles, minzoom=1, maxzoom=16)

In [ ]:
tilefile_name = "jpl_tile_test.json"
with MosaicBackend(tilefile_name, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

In [ ]:
mosaic_file_location = upload_and_return_location([tilefile_name])
mosaic_file_location

In [ ]:
import ipycmc

In [ ]:
w = ipycmc.MapCMC()
w

In [ ]:
# Build a WMTS call
"""
All of this is subject to change in a future version
The important parameters for users:
  url : the S3 path to the MosaicJSON file,
  bidx (band number), 
  rescale (required if using non Byte data type), 
  colormap_name or colormap

Other parameters are possible, see https://titiler.maap-project.org/docs#/MosaicJSON/wmts_mosaicjson_WMTSCapabilities_xml_get
"""

wmts_url = f"x{titiler_url}/mosaicjson/WMTSCapabilities.xml"
params = {
    "tile_format": "png",
    "tile_scale": "1",
    "pixel_selection": "first",
    "TileMatrixSetId": "WebMercatorQuad",
    "url": mosaic_file_location[0],
    "bidx": "1", # Select which band to use
    "resampling_method":"nearest",
    "rescale": "0,1000", # Values in data are from 0 to 1
    "return_mask": "true",
    "colormap_name": "viridis" # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

In [ ]:
wmts_link = wmts_call.replace('xhttps', 'https')
w.load_layer_config(wmts_link, "wmts/xml")

## Add your mosaic to dataset.json file

Follow the same steps above for **Create your dashboard dataset json** however the `tiles_link` should start with `{titiler_server_url}/mosaicjson/tiles/WebMercatorQuad/{z}/{x}/{y}.png?`